In this notebook, we use the [document API](https://api.bnf.fr/fr/api-document-de-gallica#scroll-nav__10__2) from Gallica to get the text that are surrounding our illustrations. To do this, we will first need to find back the Gallica IDs from the documents that have at least one illustration, and then we will query their API and convert the documents into the needed format for later topic modelling.

### Retrieving the Gallica documents

In [233]:
# Some imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from collections import Counter

In [234]:
# Data file, the same used for Vikus Viewer
data = pd.read_csv("data/data.csv")

In [82]:
# A function to query the Gallica API
# And parse the response to only get the text
def from_soup_to_string(request_url):
    response = requests.get(request_url) # Request API
    soup = BeautifulSoup(response.content, 'html.parser') # Parse response
    soup_str = soup.prettify()
    tmp = soup_str[soup_str.index("<hr/>") + 5 :] # The text is between <hr/> tags
    if "<hr/>" not in tmp:
        return ""
    html_text = tmp[:tmp.index("<hr/>")]
    p_texts = BeautifulSoup(html_text, 'html.parser').find_all('p') # Extract and concatenate all the texts in <p> divs
    whole_text = ""
    for x in p_texts:
        whole_text += x.text.replace("\n", "").replace("\\", "")
    return whole_text.strip()

In [93]:
# Select Gallica subset
data_gallica = data[data["_iiif-link"].str.contains("gallica")]
dict_text = dict()
for doc in tqdm(data_gallica.iterrows()):
    # Add entry in dictionnary with the data entry ID as key, and the text as value
    # If there are multiple pages in the same data entry, concatenate them all 
    dict_text[doc[1]["id"].split("_")[1]] = dict_text.get(doc[1]["id"].split("_")[1], "") + from_soup_to_string(doc[1]["_iiif-link"] + ".texteBrut")

3499it [11:48,  4.94it/s]


In [199]:
data_gallica.sample()

,id,_description,_artist,_source,_material,_dimensions,_journal-id,_date-artwork,year,keywords,_link-dfkv,_iiif-link,_journal-name,_link-dfkv-md,_link-iiif-md
4136,ILLU_15698_150_2,NaN,NaN,NaN,NaN,NaN,1479,NaN,1912,"Reproduction, Small Illustration",https://dfkv.dfkg.org/ng/index.html#/records/1...,https://gallica.bnf.fr/ark:/12148/bpt6k6106552...,L'Art et les artistes,[HERE](https://dfkv.dfkg.org/ng/index.html#/re...,[HERE](https://gallica.bnf.fr/ark:/12148/bpt6k...


## Create Bag-of-Word files

Now that we have all the texts that are around the illustrations, we can prepare the data for further use. We first clean the texts by removing special characters, that we list bellow

In [451]:
to_remove = ["'", ",", ";", ":", ".", "!", "?", "’", "(", ")", "\"", "%", "#", "$", "&", "*", "+"]

In [452]:
def tokenize(doc):
    # Remove the special characters
    for p in to_remove:
        doc = doc.replace(p, " ")
    # Put everything in lowercase
    tokens = doc.lower().split(" ")
    # Exclde stopwords and words that have a digit or weird character in them
    tokens = [t for t in tokens if t not in fr_stop and t != '' and len(t)>1 and "^" not in t and "<" not in t and not any(map(str.isdigit, t))]
    return Counter(tokens)

In [453]:
# Tokenize each text
dict_token_docs = dict()
for k, v in dict_text.items():
    dict_token_docs[k] = tokenize(v)

In [454]:
# Vocabulary
vocab = set()
for counter in dict_token_docs.values():
    vocab = vocab.union(set(counter.keys()))

In [455]:
# Alphabetically sort the vocabulary
vocab = sorted(list(vocab))

In [456]:
# Create a new vocab list that we will use to store the actual words written in the vocab document
new_vocab = []

In [457]:
# Create vocabulary document
# Contains the list of words
# One word per line
with open('data/vocab.dfkv.txt', 'a') as f:
    f.truncate(0)
    for v in vocab:
        try:
            f.writelines(str(v) + "\n")
            new_vocab.append(v)
        except:
            vocab.remove(v)

In [462]:
# Create the docword document
with open('data/docword.dfkv.txt', 'a') as f:
    f.truncate(0)
    total_tokens = sum([sum(list(c.values())) for c in dict_token_docs.values()])
    # Header lines :
    # Number of documents
    # Number of unique words
    # Number of words in total
    f.writelines([str(len(dict_token_docs.keys())) + "\n",  str(len(new_vocab))+ "\n", str(total_tokens) + "\n"])
    for k, v in tqdm(sorted(dict_token_docs.items())):
        try:
            for w, n in v.items():
                # Each line consists of :
                # DOC_ID WORD_ID WORD_COUNT
                to_write = str(k) + " " + str((new_vocab.index(w) + 1)) + " " + str(n) + "\n"
                f.writelines(to_write)
        except:
            pass

100%|██████████| 445/445 [01:57<00:00,  3.80it/s]
